In [ ]:
import pandas as pd
import numpy as np
import os
import platform

In [ ]:
D0_data = pd.read_csv("/content/D0_colors_data.csv")
if 'Unnamed: 0' in D0_data.columns:
    D0_data = D0_data.drop(['Unnamed: 0'], axis=1)

NameError: name 'pd' is not defined

In [ ]:
D0_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Columns: 217 entries, shows to previous_slot_cume_reach_%_in_launch_week
dtypes: float64(188), int64(14), object(15)
memory usage: 61.2+ KB


# System Variables

In [ ]:
sytem_related_df = pd.DataFrame()
sytem_related_df["shows"] = D0_data["shows"]

In [ ]:
sytem_related_df["Vertical Duplication"] = D0_data["vertical_duplication_%"].fillna(0)
sytem_related_df["Previous show cume reach"] = D0_data["previous_slot_cume_reach_%_in_launch_week"].fillna(0)
sytem_related_df["Overall Channel Pre 4 weeks Reach"] = D0_data["over_allchannel_reach%_(pre_4_wks)"].fillna(0)
sytem_related_df["Slot_Pre 4 Wks Reach"] = (D0_data["prev_4_wk_avg_cum_%"]) * 100

for i in range(len(sytem_related_df["Slot_Pre 4 Wks Reach"])) :
  if sytem_related_df["Slot_Pre 4 Wks Reach"][i] >= 20 and sytem_related_df["Slot_Pre 4 Wks Reach"][i] < 100 :
    sytem_related_df["Slot_Pre 4 Wks Reach"][i] = sytem_related_df["Slot_Pre 4 Wks Reach"][i]/10

  elif sytem_related_df["Slot_Pre 4 Wks Reach"][i] >= 100 :
    sytem_related_df["Slot_Pre 4 Wks Reach"][i] = sytem_related_df["Slot_Pre 4 Wks Reach"][i]/100
  else:
    sytem_related_df["Slot_Pre 4 Wks Reach"][i] = sytem_related_df["Slot_Pre 4 Wks Reach"][i]

sytem_related_df["Lead out"] = D0_data["launch_week_lead_out_–_for_the_slot_after_the_launch_slot_cumm_rch%"].fillna(0)

<ipython-input-45-d4e73bf58da6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sytem_related_df["Slot_Pre 4 Wks Reach"][i] = sytem_related_df["Slot_Pre 4 Wks Reach"][i]
<ipython-input-45-d4e73bf58da6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sytem_related_df["Slot_Pre 4 Wks Reach"][i] = sytem_related_df["Slot_Pre 4 Wks Reach"][i]
<ipython-input-45-d4e73bf58da6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sytem_related

# Marketing Variables

**Marketing Through TV**

In [ ]:
## Grp's and Grp days

# Grp's
grps = ['shows', 'campaign_grps_(home)', 'campaign_grps_(nw)_paid_+_support', 'campaign_grps_(osn)']
campaign_grps_home_nw_osn = D0_data[grps].fillna(0)

##Days
campaign_grp_days = pd.DataFrame()
days_home = ["#_of_weeks_(home)"]
campaign_grp_days["Num_of_days_home"] = (D0_data[days_home]*7).astype(int)

days_nw_osn = ["#_of_days_(nw)__higher_of__support_vs_paid_considered", '#_of_days_(osn)']
campaign_grp_days[days_nw_osn]= D0_data[days_nw_osn].fillna(0)

tv_marketing_data = pd.concat([campaign_grps_home_nw_osn, campaign_grp_days], axis = 1)
desired_order_col = ["shows", "campaign_grps_(home)",  "Num_of_days_home", "campaign_grps_(nw)_paid_+_support", "#_of_days_(nw)__higher_of__support_vs_paid_considered", "campaign_grps_(osn)", "#_of_days_(osn)"]
tv_marketing_df = tv_marketing_data[desired_order_col]

**Non TV Marketing**

In [ ]:
non_tv_marketing_df = pd.DataFrame()
non_tv_marketing_df["shows"] = D0_data["shows"]

In [ ]:
## Print
print = D0_data['print__display'].fillna(0)
non_tv_marketing_df["Print"] = print

In [ ]:
## dth
dth_cols = ["dth__dlc",	"dth__others",	"cable"]
dth_df = D0_data[dth_cols].fillna(0)
dth = dth_df.sum(axis = 1)
non_tv_marketing_df["DTH"] = dth

In [ ]:
## Radio
Radio = D0_data['radio'].fillna(0)
non_tv_marketing_df["Radio"] = Radio

In [ ]:
## ooh
ooh = D0_data.filter(like = "ooh").fillna(0).sum(axis = 1)
non_tv_marketing_df["OOH"] = ooh

In [ ]:
## OGA
OGA = D0_data.filter(regex = "onground|on_ground").fillna(0).sum(axis = 1)
non_tv_marketing_df["OGA"] = OGA

In [ ]:
## all views
all_views_cc = ['views(social video)', 'views(ott)', 'views(GDN)', 'views(sub_mediums)', 'views(oth_glance_zapr)']
all_views = D0_data[all_views_cc].fillna(0).sum(axis = 1).astype(int)
non_tv_marketing_df["All_Views(social video/ott/GDN/sub_mediums/oth_glance_zapr)"] = all_views

In [ ]:
## t_reach
D0_data.loc[D0_data['shows'] == "junooniyat", "total_reach"] = 71487470
D0_data.loc[D0_data['shows'] == "neerja_ek_nayee_pehchaan", "total_reach"] = 49102656
own_reach = D0_data['total_reach'].fillna(np.mean(D0_data['total_reach'])).astype("float")
non_tv_marketing_df["Own_Media_Reach"] = own_reach

In [ ]:
## pr_reach
pr_reach = D0_data['total_reach_(from_prebuzz_to_launch)'].fillna(np.mean(D0_data['total_reach_(from_prebuzz_to_launch)']))
non_tv_marketing_df["Pr_Reach"] = pr_reach

**marketing_df**

In [ ]:
marketing_df = pd.merge(tv_marketing_df, non_tv_marketing_df, on="shows")

# show Related Variables


In [ ]:
show_related_df = pd.DataFrame()
show_related_df['shows'] = D0_data['shows']

In [ ]:
## Non fiction to fiction
D0_data['replaced_genre'] = D0_data['replaced_genre'].fillna("No")
D0_data['nf_to_f'] = np.where(D0_data['replaced_genre'].str.lower().str.contains("reality"), 1, 0)
show_related_df = pd.concat([show_related_df, D0_data['nf_to_f']], axis=1)

In [ ]:
## Leads Scale

D0_data["prominent_leads_rating_scale"] = D0_data["prominent_leads_rating_scale"].fillna(0)
D0_data['leads_scale'] = D0_data['prominent_leads_rating_scale'].apply(lambda x: '1' if x in [0, 1] else '2' if x in [3, 4] else '3' if x == 5 else None)
leads_scale_dummies = pd.get_dummies(D0_data['leads_scale'], prefix='Leads Scale', prefix_sep = " ").astype(int)
show_related_df = pd.concat([show_related_df, leads_scale_dummies], axis=1)

In [ ]:
## Genre
genre_dummies = pd.get_dummies(D0_data['genre'], prefix='Genre', prefix_sep = " ").astype(int)
show_related_df = pd.concat([show_related_df, genre_dummies], axis=1)

In [ ]:
## Launch Day
D0_data['day_of_week_of_launch_episode'] = D0_data['day_of_week_of_launch_episode'].fillna("Mon")
launch_day_dummies = pd.get_dummies(D0_data['day_of_week_of_launch_episode'], prefix='Launchday', prefix_sep = " ").astype("int")
show_related_df = pd.concat([show_related_df, launch_day_dummies], axis=1)

# Spends

In [ ]:
## total spends

spends_df = pd.DataFrame()
spends_df["shows"] = D0_data["shows"]
spends_col = ["tv__nw_spends", "tv__osn_spends", "overall_digital__spends"]
spends_df[spends_col] = D0_data[spends_col].fillna(0)

# Reach and target Variable

In [ ]:
## Reach

reach_df = pd.DataFrame()
reach_df["shows"] = D0_data["shows"]
reach_col = ["target_lw_avg_reach%", "actual_lw_avg_reach%", "actual_lw_cume_reach%"]
reach_df[reach_col] = D0_data[reach_col]

# D1 Data

In [ ]:
d1_dfs = [sytem_related_df, marketing_df, show_related_df, spends_df, reach_df]
D1_temp = pd.DataFrame({"shows": D0_data["shows"]})
for i in d1_dfs :
  D1_temp = pd.merge(D1_temp, i, on="shows")

In [ ]:
D1_temp1 = D1_temp.copy()

In [ ]:
D1_temp1.rename(columns = {"campaign_grps_(home)" : "Home GRPs", "Num_of_days_home":"Home Promo Days", "campaign_grps_(nw)_paid_+_support":"Network GRPs",
                         '#_of_days_(nw)__higher_of__support_vs_paid_considered':"Network Promo Days", "campaign_grps_(osn)" :"Outside Network GRPs", "#_of_days_(osn)" : "Outside Network Promo Days",
                          "tv__nw_spends": "tv Network Spends", "tv__osn_spends" : "Outside Network Spends", "overall_digital__spends" : "Overall Digital Spends", "target_lw_avg_reach%" : "target(lw) avg Reach", "actual_lw_avg_reach%":"actual(lw) avg Reach", "actual_lw_cume_reach%" : "actual cume reach"}, inplace = True)

In [ ]:
columns_for_d1 = ['shows', 'Vertical Duplication', 'Previous show cume reach',
       'Overall Channel Pre 4 weeks Reach', 'Slot_Pre 4 Wks Reach', 'Lead out',
       'Home GRPs', 'Home Promo Days', 'Network GRPs', 'Network Promo Days',
       'Outside Network GRPs', 'Outside Network Promo Days', 'Print', 'DTH',
       'Radio', 'OOH', 'OGA',
       'All_Views(social video/ott/GDN/sub_mediums/oth_glance_zapr)',
       'Own_Media_Reach', 'Pr_Reach', 'nf_to_f', 'Leads Scale 1',
       'Leads Scale 2', 'Leads Scale 3', 'Genre Family Drama',
       'Genre Mytho / Historical / Fantasy', 'Genre Romance',
       'Genre Social Issue', 'Launchday Mon', 'Launchday Sat', 'Launchday Sun',
       'Launchday Tue', 'Launchday Wed', 'actual cume reach']

D1 = D1_temp1[columns_for_d1]
D1["Non_tv_marketing"] = D1["Print"] + D1["DTH"] + D1["Radio"] + D1["OOH"] + D1["OGA"]
D1["GRPs"] = D1["Home GRPs"] + D1["Network GRPs"] + D1["Outside Network GRPs"]

def apply_formula(value):
    return np.log(value+1)

columns_to_transform = ['GRPs', 'Home Promo Days',  'Home GRPs',  'Network GRPs',  'Outside Network GRPs', 'Print', 'DTH', 'Radio', 'OOH', 'OGA',
                        'Non_tv_marketing', 'All_Views(social video/ott/GDN/sub_mediums/oth_glance_zapr)']

for column in columns_to_transform:
    D1[column] = D1[column].apply(apply_formula)

D1 = D1[[col for col in D1.columns if col != "actual cume reach"] + ["actual cume reach"]]

D1.to_csv("Data_for_model.csv", index = False)

<ipython-input-67-56409ad2a5d8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  D1["Non_tv_marketing"] = D1["Print"] + D1["DTH"] + D1["Radio"] + D1["OOH"] + D1["OGA"]
<ipython-input-67-56409ad2a5d8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  D1["GRPs"] = D1["Home GRPs"] + D1["Network GRPs"] + D1["Outside Network GRPs"]
<ipython-input-67-56409ad2a5d8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

# Data For Scenario Planner

In [ ]:
data_for_scenario_planner  = D1_temp1.T.reset_index(drop=True)
data_for_scenario_planner.columns = data_for_scenario_planner.iloc[0]
data_for_scenario_planner["Features"] = list(D1_temp1.columns)
data_for_scenario_planner = data_for_scenario_planner[["Features"] + [col for col in data_for_scenario_planner.columns if col != "Features"]].reset_index(drop = True)
data_for_scenario_planner.drop(data_for_scenario_planner.index[0], inplace=True)

In [ ]:
mean = {}
std = {}
for col in D1_temp1.columns:
  if D1_temp1[col].dtype == "float" or D1_temp1[col].dtype == "int" :
    mean[col]= D1_temp1[col].mean()
    std[col] = D1_temp1[col].std()
  else :
    continue
mean_df = pd.DataFrame(mean.items(), columns = ["Features", "Mean"])
std_df = pd.DataFrame(std.items(), columns = ["Features", "STD"])

In [ ]:
data_for_scenario_planner = pd.merge(data_for_scenario_planner, mean_df, on="Features")
data_for_scenario_planner = pd.merge(data_for_scenario_planner, std_df, on="Features")
data_for_scenario_planner = data_for_scenario_planner[["Features", "STD", "Mean"] + [col for col in data_for_scenario_planner.columns if col not in ["Features", "STD", "Mean"]]].reset_index(drop = True)

In [ ]:
from openpyxl import load_workbook

In [ ]:
file_path = 'Data For Scenario Planner.xlsx'

with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    data_for_scenario_planner.to_excel(writer, sheet_name='Sheet1', index=False)

In [ ]:
# numerical_columns = D0_data.select_dtypes(include=[np.number]).columns
# D0_data[numerical_columns] = D0_data[numerical_columns].fillna(0)